In [1]:

#Use slurm magic to look at the status of the Burst Buffer right now

%scontrol show burst

u'Name=cray DefaultPool=wlm_pool Granularity=82496MiB TotalSpace=1192325GiB FreeSpace=982279872MiB UsedSpace=156329920MiB\n  AltPoolName[0]=dev_pool Granularity=20624MiB TotalSpace=47693GiB FreeSpace=47693GiB UsedSpace=0\n  AltPoolName[1]=sm_pool Granularity=20624MiB TotalSpace=476930GiB FreeSpace=476166912MiB UsedSpace=0\n  Flags=EnablePersistent,TeardownFailure\n  StageInTimeout=86400 StageOutTimeout=86400 ValidateTimeout=5 OtherTimeout=300\n  GetSysState=/opt/cray/dw_wlm/default/bin/dw_wlm_cli\n  Allocated Buffers:\n    Name=build_cms_000 CreateTime=2017-11-02T14:32:19 Pool=wlm_pool Size=82496MiB State=allocated UserID=jmeppley(71721)\n    Name=build_cms_002 CreateTime=2017-11-02T13:31:42 Pool=wlm_pool Size=82496MiB State=allocated UserID=jmeppley(71721)\n    Name=build_cms CreateTime=2017-11-01T14:31:14 Pool=wlm_pool Size=82496MiB State=allocated UserID=jmeppley(71721)\n    Name=NCBI_DB2 CreateTime=2017-10-17T11:20:55 Pool=wlm_pool Size=2392384MiB State=allocated UserID=syao(30821)

In [2]:

#Reformat this to pull out a few lines that describe the configuration of the available pools

sc = %scontrol show burst
cols = sc.split("\n")
for c in range(len(cols)):
    if "Granularity" in cols[c]:
        print cols[c]


Name=cray DefaultPool=wlm_pool Granularity=82496MiB TotalSpace=1192325GiB FreeSpace=982279872MiB UsedSpace=156329920MiB
  AltPoolName[0]=dev_pool Granularity=20624MiB TotalSpace=47693GiB FreeSpace=47693GiB UsedSpace=0
  AltPoolName[1]=sm_pool Granularity=20624MiB TotalSpace=476930GiB FreeSpace=476166912MiB UsedSpace=0


In [ ]:


### Submit a job using slurm magic, requesting a BB scratch allocation. 



In [91]:
%%sbatch -t 1 -N 1 -C knl --reservation=sc17_bb
#!/bin/bash
#DW jobdw capacity=1.5TB access_mode=striped type=scratch pool=wlm_pool
echo 'allocation mount point: '
echo $DW_JOB_STRIPED

u'Submitted batch job 6900035\n'

In [4]:

### Have a look at this in scontrol:

sc = %scontrol show burst
cols = sc.split("\n")
for c in range(len(cols)):
    if "djbard" in cols[c]:
        print cols[c]

    Name=djb CreateTime=2017-11-07T10:37:00 Pool=wlm_pool Size=329984MiB State=allocated UserID=djbard(61692)
    Name=StF CreateTime=2017-09-19T11:58:42 Pool=(null) Size=2887360MiB State=allocated UserID=djbard(61692)
    UserID=djbard(61692) Used=3217344MiB


In [3]:

### Use slurm magic to see what job is in the queue

%squeue -u djbard 

,JOBID,PARTITION,NAME,USER,ST,TIME,NODES,NODELIST(REASON)


In [ ]:


### Create a persistent reservation



In [95]:
%%sbatch -p special -t 1 -N 1 -C knl  --reservation=sc17_bb
#!/bin/bash
#BB create_persistent name=myPR capacity=300GB access_mode=striped type=scratch 

u'Submitted batch job 6900072\n'

In [96]:

### User slurm magic to check my job status

%squeue -u djbard 


,JOBID,PARTITION,NAME,USER,ST,TIME,NODES,NODELIST(REASON)
0,6900072,special,sbatch,djbard,PD,0:00,1,(BurstBufferResources)


In [97]:

## Have a look at my allocations using scontrol

sc = %scontrol show burst
cols = sc.split("\n")
for c in range(len(cols)):
    if "djbard" in cols[c]:
        print cols[c]

    Name=myPR CreateTime=2017-09-19T10:05:52 Pool=wlm_pool Size=329984MiB State=allocated UserID=djbard(61692)
    UserID=djbard(61692) Used=329984MiB


In [98]:

### Have a look at the output log

!ls slurm*


slurm-6900035.out


In [99]:
!cat slurm-6900035.out

allocation mount point: 
/var/opt/cray/dws/mounts/batch/6900035_striped_scratch/


In [ ]:


#### Do something a bit more complicated: use my persistent reservation, 
### and request an additional scratch space. 
### stage_in a directory to the persistent allocation
### stage_in a file to the scratch allocation




In [105]:

%%sbatch -p special -t 1 -N 1 -C knl  --reservation=sc17_bb
#!/bin/bash
#DW persistentdw name=myPR
#DW stage_in source=/global/cscratch1/sd/djbard/data  destination=$DW_PERSISTENT_STRIPED_myPR/mydatadir  type=directory


#DW jobdw capacity=500GB access_mode=striped type=scratch pool=wlm_pool
#DW stage_in source=/global/cscratch1/sd/djbard/test1Gb.db  destination=$DW_JOB_STRIPED/my1GBfile.db type=file


echo 'PR allocation mount point: '
echo $DW_PERSISTENT_STRIPED_myPR

echo 'scratch allocation mount point: '
echo $DW_JOB_STRIPED

echo 'contents of PR:'
ls -l $DW_PERSISTENT_STRIPED_myPR
ls -l $DW_PERSISTENT_STRIPED_myPR/mydatadir

echo 'contents of scratch alloction:'
ls -l $DW_JOB_STRIPED

u'Submitted batch job 6901084\n'

In [102]:

### Check the job is in the queue....

%squeue -u djbard 

,JOBID,PARTITION,NAME,USER,ST,TIME,NODES,NODELIST(REASON)
0,6900138,special,sbatch,djbard,PD,0:00,1,(BurstBufferResources)


In [107]:

### Have a look at the output file

!cat slurm-6901084.out

cat: slurm-6901084.out: No such file or directory


In [ ]:
## Delete my persstent reservation

In [104]:
%%sbatch -p special -t 1 -N 1 -C knl  --reservation=sc17_bb
#!/bin/bash
#BB destroy_persistent name=myPR 

u'Submitted batch job 6900351\n'